In [ ]:
%pip install emoji
%pip install SentencePiece
%pip install transformers
%pip install pandas
%pip install nltk
%pip install datasets evaluate transformers[sentencepiece]
%pip install cuda-python
%pip3 install -U scikit-learn scipy matplotlib
%pip install pytorch-accelerated
%pip install datasets evaluate transformers[sentencepiece]
%pip install accelerate -U
%pip install transformers[torch]
%pip install torch
%pip install scikit-learn

CREATE DATA
-

In [ ]:
import pandas as pd
import emoji
import re
import string
import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import transformers
import torch
from collections import Counter
nltk.download('omw-1.4')

In [4]:
data_real = pd.read_csv(r"C:\Users\Askar\Documents\GitHub\AI-tasks\Fake_News_Football\Data\real.csv")
data_fake = pd.read_csv(r"C:\Users\Askar\Documents\GitHub\AI-tasks\Fake_News_Football\Data\fake.csv")
data_fake["label"] = 0
data_real["label"] = 1
df = pd.concat([data_fake,data_real],ignore_index=True)
df=df.dropna()
df = df.reset_index(drop=True)
df

,tweet,label
0,"the tongue of his condition, now i saw things ...",0
1,"by god the great, after i saw the derby of wyd...",0
2,"believe in god, this zamalek fans are as good ...",0
3,the coach appeared in the media and told the t...,0
4,asno smito\r\nwhoever knows her tells me a mas...,0
...,...,...
41846,shawky gharib: we excluded 4 players from the ...,1
41847,dina musharraf resumes her career in the singa...,1
41848,"indeed, al-ahly ended its agreement with ricar...",1
41849,"hazem imam, the captain of zamalek, decided to...",1


In [5]:
"""
CLEAN TEXT
"""

#Clean emojis from text
def strip_emoji(text):
    return emoji.demojize(text, language='en') #remove emoji
    
#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(@|https?\:\/\/)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

def stop_words(text):
    # Download stop-words model
    nltk.download('punkt')
    nltk.download('stopwords')

    stop_words = set(stopwords.words('english')) 

    # Tokenize
    tokens = word_tokenize(text)

    # Delete stop words
    filtered_tokens = [word for word in tokens if word.casefold() not in stop_words]

    # Return result
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def lemmatize(text):
    # Download model
    nltk.download('punkt')
    nltk.download('wordnet')

    # Create lemmatize object
    lemmatizer = WordNetLemmatizer()

    # Tokenize sentence
    tokens = nltk.word_tokenize(text)

    # Lemmatize words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]

    # Return result
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

In [ ]:
for text in range(0,len(df)):
    df["tweet"][text] = lemmatize(stop_words(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(df["tweet"][text])))))))

In [ ]:
"""Convert to list"""
data = df.values.tolist()
data_list = [[data[i][0],data[i][1]] for i in range(len(data))]
text = [[data[i][0]] for i in range(len(data))]
text[:7]

In [9]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class DataFrameDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        text = self.data.iloc[index]['tweet']
        label = self.data.iloc[index]['label']     
        return text, label


# Creating an instance of the Datafromdataset class
dataset = DataFrameDataset(df)

# Creating a DataLoader using dataset and the collate_fn argument to apply padding
dataloader = DataLoader(dataset,batch_size=len(dataset),shuffle=True)

tokenize_data=[]
input_ids, labels = next(iter(dataloader))
tokenize_data=list(zip(input_ids, labels))


In [10]:
from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict
import datasets
"""
In this part we will create 2 topics:
- train
- test
"""
train, test = train_test_split(tokenize_data, test_size=0.4)
""""
We convert our list to DatasetDict.
"""
def convert_to_dict(train_data=None, test_data=None):
    dict_data = []

    if train_data is not None:
        for i in range(len(train_data)):
            dct = {
                "idx": i,
                "labels": train_data[i][1],
                "sentence": train_data[i][0]
            }
            dict_data.append(dct)

    if test_data is not None:
        for i in range(len(test_data)):
            dct = {
                "idx": i,
                "labels": test_data[i][1],
                "sentence": test_data[i][0]
            }
            dict_data.append(dct)

    dataset = Dataset.from_dict({
        "idx": [dict_data[i]["idx"] for i in range(len(dict_data))],
        "sentence": [dict_data[i]["sentence"] for i in range(len(dict_data))],
        "labels": [dict_data[i]["labels"] for i in range(len(dict_data))]
    })

    dataset_dict = DatasetDict()
    if train_data is not None:
        dataset_dict["train"] = dataset
    if test_data is not None:
        dataset_dict["test"] = dataset

    return dataset_dict

combined_data = convert_to_dict(train, test)


In [11]:
from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict
import datasets


""""
We convert our list to DatasetDict.
"""
def convert_to_dict(text,type_of_dataset:str):

    if type_of_dataset.lower() != "train" and type_of_dataset.lower() != "test":
        raise ValueError('It is necessary to set the parameter (train or test).')
    
    """
    First of all, we set the type of dictionary
    """

    dict_data = []

    for i in range(len(text)):
        dct = {
            "idx": i,
            "labels": text[i][1],
            "sentence":text[i][0] 
            }
        dict_data.append(dct)

    """Here we convert our list in Dataset and after we transmit to 
    DatasetDict, where do we choose the type of our dataset part. (Train or Test)"""
    dataset = Dataset.from_dict({"idx": [dict_data[i]["idx"] for i in range(len(dict_data))],
                                'sentence': [dict_data[i]["sentence"] for i in range(len(dict_data))], 
                                "labels":[dict_data[i]["labels"] for i in range(len(dict_data))]})
    
    dataset_dict = DatasetDict({type_of_dataset: dataset})

            
    return dataset_dict

combined_data = convert_to_dict(train,"train")
test_data = convert_to_dict(test,"test")
combined_data.update(test_data)
combined_data


DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'labels'],
        num_rows: 25110
    })
    test: Dataset({
        features: ['idx', 'sentence', 'labels'],
        num_rows: 16741
    })
})

Transformers AutoModelForSequenceClassification
-
Score = 0.96
-

In [ ]:
from transformers import DataCollatorWithPadding
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

train_dt = combined_data["train"]
checkpoint = "bert-base-uncased"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(train_dt):
    return tokenizer(train_dt["sentence"], truncation=True)


tokenized_datasets = combined_data.map(tokenize_function,batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
from transformers import Trainer
from transformers import TrainingArguments
import evaluate
import numpy as np


def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

{'loss': 0.3014, 'learning_rate': 4.734522671763831e-05, 'epoch': 0.16}


 11%|█         | 1000/9417 [00:51<07:02, 19.93it/s]

{'loss': 0.2502, 'learning_rate': 4.469045343527663e-05, 'epoch': 0.32}


 16%|█▌        | 1500/9417 [01:17<06:29, 20.34it/s]

...

 96%|█████████▌| 9000/9417 [08:50<00:20, 20.22it/s]

{'loss': 0.0845, 'learning_rate': 2.2140809174896463e-06, 'epoch': 2.87}


                                                   
100%|██████████| 9417/9417 [09:42<00:00, 16.18it/s]

{'eval_loss': 0.18132726848125458, 'eval_accuracy': 0.960516098202019, 'eval_f1': 0.9625644220422496, 'eval_runtime': 28.9627, 'eval_samples_per_second': 578.018, 'eval_steps_per_second': 72.265, 'epoch': 3.0}
{'train_runtime': 582.0623, 'train_samples_per_second': 129.419, 'train_steps_per_second': 16.179, 'train_loss': 0.15499206782779812, 'epoch': 3.0}


TrainOutput(global_step=9417, training_loss=0.15499206782779812, metrics={'train_runtime': 582.0623, 'train_samples_per_second': 129.419, 'train_steps_per_second': 16.179, 'train_loss': 0.15499206782779812, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])

preds = np.argmax(predictions.predictions, axis=-1)

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

  0%|          | 0/2093 [00:00<?, ?it/s]

100%|██████████| 2093/2093 [00:28<00:00, 73.06it/s]


{'accuracy': 0.960516098202019, 'f1': 0.9625644220422496}

Neural Network With 0
-
SCORE = 0.96
-

In [12]:
from transformers import DataCollatorWithPadding
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

train_dt = combined_data["train"]
checkpoint = "bert-base-uncased"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(train_dt):
    return tokenizer(train_dt["sentence"], truncation=True)


tokenized_datasets = combined_data.map(tokenize_function,batched=True)
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [13]:
import torch
from torch.utils.data import TensorDataset
def padding_data(type_of_data:str):
    # Преобразование input_ids в формат списка тензоров
    input_ids = [torch.tensor(ids) for ids in tokenized_datasets[type_of_data]['input_ids']]

    # Применение padding к input_ids
    padded_input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True).to(device)

    # Получение меток (labels)
    labels = torch.tensor(tokenized_datasets[type_of_data]['labels'], dtype=torch.long).to(device)

    # Создание TensorDataset, объединяющего паддингованные input_ids и метки
    dataset = TensorDataset(padded_input_ids, labels)

    batch_size = 32  # Установите желаемый размер батча

    dataset = TensorDataset(padded_input_ids, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

dataloader_train = padding_data("train")
dataloader_test = padding_data("test")

In [72]:
import torch
import torch.nn as nn
from torch.optim import AdamW
import torch.nn.functional as F


class NeuralClassification(nn.Module):
    def __init__(self):
        super(NeuralClassification, self).__init__()
        self.embedding = nn.Embedding(30522, 256)#Размер словаря, размерность вектора эмбеддинга
        self.conv2 = nn.Conv2d(1, 128, kernel_size=(130,256))#Рамзер входа, размер выхода, рамзер окна 
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(128, 2) 


    def forward(self, x):
        embedded = self.embedding(x)
        embedded = embedded.unsqueeze(1)# Перестановка размерностей
        conv = self.conv2(embedded)
        conv = self.dropout(conv)
        conv = F.relu(conv)
        conv = torch.max(conv, dim=2)[0]  # Применение max pooling по временной размерности
        conv = conv.view(conv.size(0), -1)
        output = self.fc(conv)
        return output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classif_model = NeuralClassification().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(classif_model.parameters(), lr=0.001, weight_decay=0.01)



In [73]:
from tqdm import tqdm

# Определение модели, оптимизатора, функции потерь и т. д.
num_epochs = 200
# Итерация по эпохам
for epoch in range(num_epochs):
    # Обучение модели
    classif_model.train()
    total_loss = 0

    # Используем tqdm для создания прогресс-бара
    progress_bar = tqdm(dataloader_train, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    for inputs, labels in progress_bar:
        optimizer.zero_grad()
        outputs = classif_model(inputs)
        labels = labels.squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Обновляем информацию в прогресс-баре
        progress_bar.set_postfix({'loss': loss.item()})

    average_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}')

# Код для валидации модели и т. д.



Epoch 1/200, Loss: 646.1539443135262


Epoch 2/200, Loss: 543.4293029904366


Epoch 3/200, Loss: 545.6067113280296


Epoch 4/200, Loss: 543.1823194622993


Epoch 5/200, Loss: 548.7868717908859


...

Epoch 196/200, Loss: 0.27211018658440955


Epoch 197/200, Loss: 0.2576433276769068


Epoch 198/200, Loss: 0.2610355205249393


Epoch 199/200, Loss: 0.38036860600748956


Epoch 200/200, Loss: 1.6128542904200254


In [74]:
from sklearn.metrics import accuracy_score, f1_score

with torch.no_grad():
    inputs = []
    labels = []
    for batch in dataloader_test:
        input_batch, label_batch = batch
        inputs.append(input_batch.to(device))  # Move inputs to the GPU
        labels.append(label_batch.to(device))  # Move labels to the GPU

    inputs = torch.cat(inputs, dim=0)
    labels = torch.cat(labels, dim=0)

    predictions = classif_model(inputs).cpu()

# Преобразование предсказаний в метки классов
predicted_labels = torch.argmax(predictions, dim=1).cpu()

# Получение истинных меток классов
true_labels = labels.cpu()

# Вычисление accuracy и f1-score
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1-Score: {f1:.4f}')



Accuracy: 0.9404
F1-Score: 0.9404
